In [187]:
from solcx import compile_standard, install_solc
import json
from web3 import Web3
import random
from hexbytes import HexBytes

# Test loading address and private key

In [188]:
def decimalToHexString(arrayDecimal):
    return '0x'+ ''.join(map(lambda x: "0"+hex(x)[2:] if (len(hex(x)[2:])==1) else hex(x)[2:].lower() , arrayDecimal))


def loadPrivateKey(address, account_data):
    return decimalToHexString(account_data['addresses'][address.lower()]['secretKey']['data'])

def passBlocks(n):
    for i in range(n):
        w3.eth.send_transaction({
            'to': w3.eth.accounts[0],
            'from': w3.eth.accounts[0],
            'value': 1
        })
        global nonce
        nonce += 1
        
        

In [189]:
#  listen to port 7545 for ganache

w3 = Web3( Web3.HTTPProvider('http://127.0.0.1:8545'))

In [190]:
# get the first account address
print(w3.eth.accounts[0])

0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1


In [191]:
# get the first private key of account 0
account_data = open('keys.json')
account_data = json.load(account_data)
print(loadPrivateKey(w3.eth.accounts[0], account_data))

0x4f3edf983ac636a65a842ce7c78d9aa706d3b113bce9c46f30d7d21715b23b1d


In [192]:
print(HexBytes(0))

b'\x00'


# Test deploying RTDS

In [193]:
# Variable 
numberOfAccounts = 100

my_address = "0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1"
my_private_key = str(loadPrivateKey(my_address, account_data))

successTransactionNumber = 0
unSuccessTransactionNumber = 0
overallGassFee = 0
# https://solveforums.msomimaktaba.com/threads/solved-how-to-pass-a-struct-as-an-argument-in-call-using-web3-py.619080/

'''
    struct Transaction {
        address senderAddress;
        address receiverAddress;
        uint256 amount;
        uint256 transactionTime;
        bytes32 transactionChallenge;
        bool verified;
    }
'''

'''

        struct Account {

        address accountAddress;
        bool isRegistered;
        uint256 currentBalance;
        uint256 latestVerifiedBlockNumber;
        uint256 latestVerifiedTime;
        bytes32 registrationChallenge;
        bytes registeredPublicKey;
        bytes keyHandle;
        // delayed transactions
        Transaction[] _delayedTransactions;
    }

'''
#transactionTest = {'senderAddress':w3.eth.accounts[0], 'receiverAddress':w3.eth.accounts[1], 'amount':250, 'transactionTime': 1649505951, 'transactionChallenge':HexBytes(0), 'verified': False};
#senderAccount = {'accountAddress':w3.eth.accounts[0], 'isRegistered':True, 'currentBalance':2000000000,  'latestVerifiedBlockNumber':0, 'latestVerifiedTime':1649505949, 'registrationChallenge':HexBytes(0), 'registeredPublicKey': HexBytes(0), 'keyHandle':HexBytes(0), '_delayedTransactions':[]}

#receiverAccount = {'accountAddress':w3.eth.accounts[1], 'isRegistered':True, 'currentBalance':2000000000,  'latestVerifiedBlockNumber':0, 'latestVerifiedTime':1649505949, 'registrationChallenge':HexBytes(0), 'registeredPublicKey': HexBytes(0), 'keyHandle':HexBytes(0), '_delayedTransactions':''}

'\n\n        struct Account {\n\n        address accountAddress;\n        bool isRegistered;\n        uint256 currentBalance;\n        uint256 latestVerifiedBlockNumber;\n        uint256 latestVerifiedTime;\n        bytes32 registrationChallenge;\n        bytes registeredPublicKey;\n        bytes keyHandle;\n        // delayed transactions\n        Transaction[] _delayedTransactions;\n    }\n\n'

In [203]:
with open("./RTDS.sol", "r") as file:
    riskyTransactionDector = file.read()
# Compile the solidity
install_solc("0.8.7")


compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"RTDS.sol": {"content": riskyTransactionDector}},
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]}
            }
        },
    },allow_paths = ".",
    solc_version="0.8.7",
)

with open("compiled_RTDS.json", "w") as file:
    json.dump(compiled_sol, file)

# sets chain_id.
chain_id = w3.eth.chain_id
    
# sets bytecode version.
bytecode = compiled_sol["contracts"]["RTDS.sol"]["RTDS"]["evm"]["bytecode"]["object"]

# gets abi.
abi = compiled_sol["contracts"]["RTDS.sol"]["RTDS"]["abi"]

# gets bytecode version.
bytecode = compiled_sol["contracts"]["RTDS.sol"]["RTDS"]["evm"]["bytecode"]["object"]

# gets abi.
abi = compiled_sol["contracts"]["RTDS.sol"]["RTDS"]["abi"]

riskyTransactionDetector = w3.eth.contract(abi=abi, bytecode=bytecode)

# gets the latest transaction.
nonceList = [0]*numberOfAccounts
nonce = w3.eth.getTransactionCount(my_address)


# Two ways of interact with the contracts
# 1. Call: Simulate making the call and getting a return value
# 2. Transact:  actually make a state change

successTransactionNumber = 0
unSuccessTransactionNumber = 0
overallGassFee = 0

# 1.  builds a transaction.
transaction = riskyTransactionDetector.constructor().buildTransaction(
    {"chainId": chain_id, "gasPrice": w3.eth.gas_price, "from": my_address, "nonce": nonce})
nonce += 1
# 2. signs a transaction.
signed_txn = w3.eth.account.sign_transaction( transaction, private_key = my_private_key)

# 3. sends a transaction.
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

# 4. waits for the receipt.
contract_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
# Working with contracts
# 1. Contract addresses
# 2. Contract ABI
risky_transaction_detector = w3.eth.contract(address = contract_receipt.contractAddress, abi=abi)



### Tests registrationUpdator(Util.Account memory newAccount)

In [204]:
newAccount = (w3.eth.accounts[1], True,2000000000, 0, 1649505949,5,HexBytes(0), HexBytes(0),HexBytes(0), [])
transactionWeb3 = risky_transaction_detector.functions.registrationUpdator(newAccount).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("cumulative amount after registration: ",risky_transaction_detector.functions.getMaximumCumulativeAmount(w3.eth.accounts[1]).call())
print("maximum allowed ratio after registration: ",risky_transaction_detector.functions.getMaximumAllowedRatio(w3.eth.accounts[1]).call())
print("maximum allowed amount after registration: ",risky_transaction_detector.functions.getMaximumAllowedAmount(w3.eth.accounts[1]).call())


cumulative amount after registration:  0
maximum allowed ratio after registration:  [1, 1000]
maximum allowed amount after registration:  10000000000000000000


# Unit Test

## Test Maximum cumulative amount limit

### Test getMaximumCumulativeAmount(address senderAccountAddress)


In [122]:
print("maximum cumulative amount: ",risky_transaction_detector.functions.getMaximumCumulativeAmount(w3.eth.accounts[1]).call())


maximum cumulative amount:  0


### Test getMaximumAllowedAmount(address senderAccountAddress) 

In [123]:
print("maximum allowed amount: ",risky_transaction_detector.functions.getMaximumAllowedAmount(w3.eth.accounts[1]).call())


maximum allowed amount:  10000000000000000000


### Test getMaximumAllowedRatio(address senderAccountAddress)

In [124]:
print("maximum allowed ratio: ",risky_transaction_detector.functions.getMaximumAllowedRatio(w3.eth.accounts[1]).call())


maximum allowed ratio:  [1, 1000]


In [125]:
print(nonce)

32


### Tests maximumCumulativeAmountLimit(address senderAccountAddress, uint256 proposedAmount)

In [78]:
print("is transaction of 999 from account allowed: ",risky_transaction_detector.functions.maximumCumulativeAmountLimit(w3.eth.accounts[1], 999).call())
print("is transaction of 1000 from account allowed: ",risky_transaction_detector.functions.maximumCumulativeAmountLimit(w3.eth.accounts[1], 1000).call())

is transaction of 999 from account allowed:  False
is transaction of 1000 from account allowed:  False


### Tests transactionUpdator

In [71]:
transactionTest = (w3.eth.accounts[1], w3.eth.accounts[2], 200,  1649505951, HexBytes(0), False);
senderAccount = (w3.eth.accounts[1], True,2000000000, 0, 1649505949,5,HexBytes(0), HexBytes(0),HexBytes(0), [])
receiverAccount  = (w3.eth.accounts[2], True,2000000000, 0, 1649505949,5,HexBytes(0), HexBytes(0),HexBytes(0), [])
transactionWeb3 = risky_transaction_detector.functions.transactionUpdator(senderAccount,receiverAccount,transactionTest).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("is transaction of 1000 from account allowed: ",risky_transaction_detector.functions.maximumCumulativeAmountLimit(w3.eth.accounts[1], 999).call())
print("is transaction of 1001 from account allowed: ",risky_transaction_detector.functions.maximumCumulativeAmountLimit(w3.eth.accounts[1], 1000).call())
print("is transaction of 799 from account allowed: ",risky_transaction_detector.functions.maximumCumulativeAmountLimit(w3.eth.accounts[1], 799).call())
print("is transaction of 800 from account allowed: ",risky_transaction_detector.functions.maximumCumulativeAmountLimit(w3.eth.accounts[1], 800).call())

is transaction of 1000 from account allowed:  False
is transaction of 1001 from account allowed:  False
is transaction of 799 from account allowed:  False
is transaction of 800 from account allowed:  False


## Test Block Passed

In [112]:
print(w3.eth.blockNumber)

3


In [113]:
passBlocks(10)

In [114]:
print(w3.eth.blockNumber)

13


## Test Maximum unauthenticated interval 

### Tests getMaximumUnauthenticatedBlockNumber(address accountAddress)

In [115]:
print("maximum unauthenticated time interval of account 1: ",risky_transaction_detector.functions.getMaximumUnauthenticatedBlockNumber(w3.eth.accounts[1]).call())

maximum unauthenticated time interval of account 1:  360


### Tests  setMaximumUnauthenticatedBlockNumber(address accountAddress, uint256 newMaximumUnauthencatedBlockNumber)

In [116]:
newMaximumUnauthenticatedBlockNumber =  400

transactionWeb3 = risky_transaction_detector.functions.setMaximumUnauthenticatedBlockNumber(w3.eth.accounts[1],newMaximumUnauthenticatedBlockNumber).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print("maximum unauthenticated time interval of account 1 after setting: ",risky_transaction_detector.functions.getMaximumUnauthenticatedBlockNumber(w3.eth.accounts[1]).call())

maximum unauthenticated time interval of account 1 after setting:  400


### Tests  maximumUnauthenticatedIntervalLimit(Util.Account memory account)

In [117]:
newAccount = (w3.eth.accounts[2], True,2000000000, 0, 1649505949,5,HexBytes(0), HexBytes(0),HexBytes(0), [])
transactionWeb3 = risky_transaction_detector.functions.registrationUpdator(newAccount).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)


In [118]:
print(risky_transaction_detector.functions.maximumUnauthenticatedIntervalLimit(newAccount).call())

False


In [79]:
passBlocks(359)
print(risky_transaction_detector.functions.maximumUnauthenticatedIntervalLimit(newAccount).call())
passBlocks(1)
print(risky_transaction_detector.functions.maximumUnauthenticatedIntervalLimit(newAccount).call())

True
True


In [119]:
newAccount = (w3.eth.accounts[2], True,2000000000, 465, 1649505949,5,HexBytes(0), HexBytes(0),HexBytes(0), [])

print(risky_transaction_detector.functions.maximumUnauthenticatedIntervalLimit(newAccount).call())


False


## Test Maximum amount to unregistered account

### Tests getMaximumAmountToUnregisteredAccount(address accountAddress)

In [120]:
print(risky_transaction_detector.functions.getMaximumAmountToUnregisteredAccount(w3.eth.accounts[1]).call())



1000000000000000


### Tests maximumAmountToUnregisteredAccountLimit(address sendeAccountAddress, bool isRegistered, uint256 amount)

In [121]:


print(risky_transaction_detector.functions.maximumAmountToUnregisteredAccountLimit(w3.eth.accounts[1], False, 999999999999999).call())
print(risky_transaction_detector.functions.maximumAmountToUnregisteredAccountLimit(w3.eth.accounts[1], False, 1000000000000000).call())



False
True


### Tests setMaximumAmountToUnregisteredAccount(address accountAddress, uint256 newMaximumAllowedAmountToUnregisteredAccount)

In [122]:
newMaximumAllowedAmountToUnregisteredAccount =  2000000000000000
transactionWeb3 = risky_transaction_detector.functions.setMaximumAmountToUnregisteredAccount(w3.eth.accounts[1],newMaximumAllowedAmountToUnregisteredAccount).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(risky_transaction_detector.functions.getMaximumAmountToUnregisteredAccount(w3.eth.accounts[1]).call())


2000000000000000


## Test Maximum frequency limit 

###  Tests getMaximumAllowedTransactionsPerThousandBlocks(address senderAccountAddress)

In [194]:
print(risky_transaction_detector.functions.getMaximumAllowedTransactionsPerThousandBlocks(w3.eth.accounts[1]).call())


0


### Tests setMaximumAllowedTransactionsPerThousandBlocks(address senderAccountAddress, uint256 newMAximumAllowedTransactionsPerThousandBlocks) 

In [204]:
newMAximumAllowedTransactionsPerThousandBlocks =  6
transactionWeb3 = risky_transaction_detector.functions.setMaximumAllowedTransactionsPerThousandBlocks(w3.eth.accounts[1], newMAximumAllowedTransactionsPerThousandBlocks ).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(risky_transaction_detector.functions.getMaximumAllowedTransactionsPerThousandBlocks(w3.eth.accounts[1]).call())

6


In [205]:
newMAximumAllowedTransactionsPerThousandBlocks =  11
transactionWeb3 = risky_transaction_detector.functions.setMaximumAllowedTransactionsPerThousandBlocks(w3.eth.accounts[1], newMAximumAllowedTransactionsPerThousandBlocks).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
try:
    tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
except:
    print("correct")
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)


correct


In [33]:
transactionWeb3 = risky_transaction_detector.functions.maximumFrequencyLimit(w3.eth.accounts[1]).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_receipt)

AttributeDict({'transactionHash': HexBytes('0xebb4296e8936437bd45c3a599cd3fb4bbb4401da526f4adb0989f126638266d5'), 'transactionIndex': 0, 'blockHash': HexBytes('0x400238a3f89e8412c5037c30d2355607c6071b65dfb6a0b1d23150bc1654df01'), 'blockNumber': 9, 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1', 'to': '0xD833215cBcc3f914bD1C9ece3EE7BF8B14f841bb', 'gasUsed': 27109, 'cumulativeGasUsed': 27109, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')})


In [138]:
transactionWeb3 = risky_transaction_detector.functions.updateRecentTransactions(w3.eth.accounts[1]).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

### Tests maximumFrequencyLimit(address senderAccountAddress)

In [186]:

for i in range(11):
    #transactionWeb3 = risky_transaction_detector.functions.popOutdatedTransactions(w3.eth.accounts[1]).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
    #nonce += 1
    #signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
    #tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
    #tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    #print(tx_receipt)
    print("number of recent transaction: ",risky_transaction_detector.functions.getRecentTransactionsNumber(w3.eth.accounts[1]).call())
    # maximumFrequencyLimit(address senderAccountAddress)
    print("number of recent transaction: ",risky_transaction_detector.functions.maximumFrequencyLimit(w3.eth.accounts[1]).call())
    transactionWeb3 = risky_transaction_detector.functions.updateRecentTransactions(w3.eth.accounts[1]).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
   
    signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
    tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    nonce += 1
    
    

number of recent transaction:  0
number of recent transaction:  False
number of recent transaction:  1
number of recent transaction:  False
number of recent transaction:  2
number of recent transaction:  False
number of recent transaction:  3
number of recent transaction:  False
number of recent transaction:  4
number of recent transaction:  False
number of recent transaction:  5
number of recent transaction:  True
number of recent transaction:  6
number of recent transaction:  True
number of recent transaction:  7
number of recent transaction:  True
number of recent transaction:  8
number of recent transaction:  True
number of recent transaction:  9
number of recent transaction:  True
number of recent transaction:  9
number of recent transaction:  True


In [158]:
print(risky_transaction_detector.functions.getRecentTransactionsNumber(w3.eth.accounts[1]).call())

8


## Tests Small Amount Release

### Tests  getSmallAmount(address senderAccountAddress) 

In [206]:
print(risky_transaction_detector.functions.getSmallAmount(w3.eth.accounts[1]).call())

1000000000


### Tests   getSmallRatio(address senderAccountAddress)

In [207]:
print(risky_transaction_detector.functions.getSmallRatio(w3.eth.accounts[1]).call())

[1, 10000]


### Tests   setSmallAmount (address senderAccountAddress, uint256 newAmount)

In [208]:
transactionWeb3 = risky_transaction_detector.functions.setSmallAmount(w3.eth.accounts[1], 1000).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
nonce += 1
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(risky_transaction_detector.functions.getSmallAmount(w3.eth.accounts[1]).call())

1000


### Tests   setSmallRatio(address senderAccountAddress, uint256[] memory newRatio) 

In [210]:
transactionWeb3 = risky_transaction_detector.functions.setSmallRatio(w3.eth.accounts[1], [1,1000]).buildTransaction({"chainId": chain_id, "from": my_address, "gasPrice": w3.eth.gas_price, "nonce":nonce, "gas": 300000})
signed_txn = w3.eth.account.sign_transaction(transactionWeb3, private_key=loadPrivateKey(w3.eth.accounts[0], account_data))
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
nonce += 1
print(risky_transaction_detector.functions.getSmallRatio(w3.eth.accounts[1]).call())

[1, 1000]


### Tests smallAmountRelease(Util.Account memory senderAccount, uint256 amount) 

In [222]:
senderAccount = (w3.eth.accounts[1], True,2000000000, 0, 1649505949,5,HexBytes(0), HexBytes(0),HexBytes(0), [])
print(risky_transaction_detector.functions.smallAmountRelease(senderAccount,1).call())
print(risky_transaction_detector.functions.smallAmountRelease(senderAccount,10000000000000000000).call())

True
False


## Test view functions

In [219]:
#transactionTest = {'senderAddress':w3.eth.accounts[0], 'receiverAddress':w3.eth.accounts[1], 'amount':250, 'transactionTime': 1649505951, 'transactionChallenge':HexBytes(0), 'verified': False};
#senderAccount = {'accountAddress':w3.eth.accounts[0], 'isRegistered':True, 'currentBalance':2000000000,  'latestVerifiedBlockNumber':0, 'latestVerifiedTime':1649505949, 'registrationChallenge':HexBytes(0), 'registeredPublicKey': HexBytes(0), 'keyHandle':HexBytes(0), '_delayedTransactions':[]}
#receiverAccount = {'accountAddress':w3.eth.accounts[1], 'isRegistered':True, 'currentBalance':2000000000,  'latestVerifiedBlockNumber':0, 'latestVerifiedTime':1649505949, 'registrationChallenge':HexBytes(0), 'registeredPublicKey': HexBytes(0), 'keyHandle':HexBytes(0), '_delayedTransactions':''}
transactionTest = (w3.eth.accounts[0], w3.eth.accounts[1], 250,  1649505951, HexBytes(0), False);
senderAccount = (w3.eth.accounts[0], True,2000000000, 0, 1649505949,5,HexBytes(0), HexBytes(0),HexBytes(0), [])
receiverAccount  = (w3.eth.accounts[1], True,2000000000, 0, 1649505949,5,HexBytes(0), HexBytes(0),HexBytes(0), [])
print(risky_transaction_detector.functions.checkSecurity(senderAccount, receiverAccount, transactionTest).call())

ContractLogicError: execution reverted: VM Exception while processing transaction: revert